In [4]:
import fitz
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis.gensim_models
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Downloading package punkt to /Users/stan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [24]:
file_path = 'ESG Reports/2022_Apple_ESG_Report.pdf'

In [25]:
def cleanup_text(text):
    # remove non-breaking spaces
    text = text.replace(u'\xa0', u' ')
    # remove bullet points
    text = text.replace(u'•', u'')
    # remove any non-alphanumeric, non-hyphen characters
    text = re.sub(r'[^A-Za-z0-9- ]', '', text)
    # remove words with hyphens, as they could be compound words
    text = re.sub(r'\w+-\w+', '', text)
    return text

In [26]:
# def text_process(text_str):
#     if ' ' in text_str:
#         text_str = text_str.replace(' ', ' ')
#     if '•' in text_str:
#         text_str = text_str.replace('•', ' ')
#     text_str = re.sub(' +', ' ', text_str)
#     if '-' in text_str:
#         text_str = text_str.replace('- ', '').replace(' -', '').replace(' ,', ',').replace(' .', '.')
#     text_str = text_str.strip()
#     return text_str

def text_process(text_str):
    # handle punctuation and special characters
    text_str = re.sub(r'[^\w\s]', '', text_str)
    if '-' in text_str:
        text_str = text_str.replace('- ', '').replace(' -', '').replace(' ,', ',').replace(' .', '.')
    text_str = text_str.strip()
    return text_str

In [29]:
all_text = []
stop_words = set(stopwords.words('english'))

with fitz.open(file_path) as doc:
    for page in doc:
        text = page.get_text()
        # text = cleanup_text(text)  # Apply cleanup_text here
        text = text.replace('•', ' ')
        text = text_process(text)
        text = text.replace('\n', ' ')
        text = text.replace('\u2003', ' ')
        text = text.strip()
        all_text.append(text)


lemmatizer = WordNetLemmatizer()

processed_text = []


for doc in all_text:
    tokens = word_tokenize(doc.lower())
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens if token not in string.punctuation]
    no_stops = [token for token in lemmatized if token not in stop_words]
    processed_text.append(" ".join(no_stops))

final_text = ' '.join(processed_text)
final_text = re.sub(r'\d+', '', final_text)
final_text = re.sub(' +', ' ', final_text)



# full_text = ' '.join(all_text).lower()
# full_text = ''.join([char for char in full_text if char not in string.punctuation])
# full_text = re.sub(' +', ' ', full_text)  # remove excessive spaces
# full_text_words = full_text.split()
# full_text_words = [cleanup_text(word) for word in full_text_words if word.strip() in full_text_words]
# full_text_words = [lemmatizer.lemmatize(word) for word in full_text_words] # add this line for lemmatization
# final_text = ' '.join(full_text_words)
# final_text = re.sub(' +', ' ', final_text)  # remove excessive spaces

In [30]:
final_text

'apple esg report environmental social governance report introduction letter tim cook report highlight approach commitment transparency advocating change commitment human right environment approach climate change resource smarter chemistry people approach inclusion diversity growth development benefit compensation engagement workplace practice policy health safety apple supplier approach labor human right supply chain health safety wellness responsible material sourcing education professional development environment customer approach privacy accessibility inclusive design education health caring customer community approach racial equity justice initiative education affordable housing initiative corporate donation employee giving job creation governance corporate governance ethic compliance tax payment appendix award recognition united nation sustainable development goal stakeholder engagement esg data summary report endnotes esg index report contains forwardlooking statement actual res

In [35]:
processed_text

['apple 2022 esg report environmental social governance report',
 'introduction 3 letter tim cook 4 report highlight 6 approach 8 commitment transparency 9 advocating change 10 commitment human right environment 13 approach 13 climate change 18 resource 20 smarter chemistry people 23 approach 23 inclusion diversity 26 growth development 27 benefit 28 compensation 29 engagement 30 workplace practice policy 33 health safety apple supplier 37 approach 40 labor human right supply chain 43 health safety wellness 44 responsible material sourcing 45 education professional development 46 environment customer 48 approach 48 privacy 50 accessibility 52 inclusive design 53 education 54 health 55 caring customer community 59 approach 60 racial equity justice initiative 62 education 64 affordable housing initiative 65 corporate donation 67 employee giving 68 job creation governance 70 corporate governance 71 ethic compliance 72 tax payment appendix 74 award recognition 75 united nation sustainable 

In [33]:
# 1. Convert into list of tokens
doc_list = [doc.split() for doc in processed_text]

# 2. Create the Dictionary and Corpus
dictionary = corpora.Dictionary(doc_list)
corpus = [dictionary.doc2bow(doc) for doc in doc_list]

# 3. Build the LDA model
lda_model = LdaModel(corpus=corpus, num_topics=30, id2word=dictionary, passes=10, random_state=1)

# 4. Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.134043  0.001363       1        1  15.070914
24     0.117203 -0.125662       2        1  10.083844
0      0.123901  0.057658       3        1   6.561443
16     0.071439  0.026654       4        1   6.506757
4      0.076903  0.088919       5        1   6.391172
19     0.147759 -0.030390       6        1   5.369205
26     0.158405  0.015624       7        1   5.192059
13     0.102026 -0.103803       8        1   5.105228
12     0.122093 -0.069575       9        1   4.972287
6      0.120543 -0.060261      10        1   4.479693
7      0.112653  0.010868      11        1   4.341894
18     0.064207  0.164972      12        1   3.577544
2      0.091885  0.081149      13        1   3.515041
28    -0.017882  0.198113      14        1   3.250376
8      0.076684 -0.000568      15        1   3.103759
25     0.070068 -0.029490      16        1   2.995264
21     0.057818 -0.082552      17        1   2.496199
10    -0.034189 -0.128962      18        1   1.505216
11    -0.033150  0.055692      19        1   1.414362
29    -0.081183  0.012914      20        1   1.172422
3     -0.105921  0.156915      21        1   0.951456
20    -0.117701 -0.077462      22        1   0.939587
23    -0.190763  0.006986      23        1   0.429398
5     -0.175411 -0.023936      24        1   0.266041
9     -0.104725 -0.029666      25        1   0.260571
17    -0.154560 -0.025061      26        1   0.009654
22    -0.147080 -0.021431      27        1   0.009654
14    -0.165458 -0.024108      28        1   0.009654
27    -0.158651 -0.021382      29        1   0.009654
15    -0.160954 -0.023520      30        1   0.009654, topic_info=           Term        Freq       Total Category  logprob  loglift
1         apple  654.000000  654.000000  Default  30.0000  30.0000
231     product  194.000000  194.000000  Default  29.0000  29.0000
135    supplier  314.000000  314.000000  Default  28.0000  28.0000
96       health   93.000000   93.000000  Default  27.0000  27.0000
308      energy   80.000000   80.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
282        also    0.000836  128.142432  Topic30  -8.1219  -2.6942
147      across    0.000830  126.823988  Topic30  -8.1294  -2.6914
272        2021    0.000829  137.064718  Topic30  -8.1309  -2.7705
105  initiative    0.000826  106.848682  Topic30  -8.1345  -2.5251
249        team    0.000822  158.529125  Topic30  -8.1391  -2.9242

[2057 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
1093      4  0.725340     0
1093     13  0.145068     0
1096     13  0.943665    02
1101      4  0.226715     1
1101      5  0.317401     1
...     ...       ...   ...
1379     11  0.115879  zero
1379     12  0.057939  zero
1379     14  0.289697  zero
1379     16  0.057939  zero
2973      2  0.607700  zoom

[4345 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 25, 1, 17, 5, 20, 27, 14, 13, 7, 8, 19, 3, 29, 9, 26, 22, 11, 12, 30, 4, 21, 24, 6, 10, 18, 23, 15, 28, 16])

In [ ]:
file_path = "ESG Reports/0a8d901480b1f390a3bb5e7bd6604d8b40ab7e0c.pdf"

all_text = []
with fitz.open(file_path) as doc:
    for page in doc:
        text = page.get_text()
        text = text.replace('•', ' ')
        text = text_process(text)
        text = text.replace('\n', ' ')
        text = text.replace('\u2003', ' ')
        text = text.strip()
        all_text.append(text)

In [ ]:
# 1. Convert into list of tokens
doc_list = [doc.split() for doc in all_text]

# 2. Create the Dictionary and Corpus
dictionary = corpora.Dictionary(doc_list)
corpus = [dictionary.doc2bow(doc) for doc in doc_list]

# 3. Build the LDA model
lda_model = LdaModel(corpus=corpus, num_topics=5, id2word=dictionary, passes=10)

# 4. Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis
